In [6]:
library(dplyr)
#Read in data and save it in df object (original data for comparison)
dataFile = '/mnt/workspace/GWTG/COVID19/data/covid19_cvd_aug20.sas7bdat'
formatsFile = '/mnt/workspace/GWTG/COVID19/data/R_Python_windows_formats.sas7bcat'

## Assigning the dataframe to the dfor object
dfor <- haven::read_sas(dataFile, formatsFile)



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [7]:
#Print Dimensions
dim(dfor)

#Print Unique Record ID
cat("Original Record, Patient, Hospital Counts")
length(unique(dfor$RCRDNUM)) 
length(unique(dfor$PATIENT_ID))
length(unique(dfor$SRC_FAC_ID))

#Exclude NA on ClinTrials 
cat("\n Missing COVCLINTRIAL")
sum(is.na(dfor$COVCLINTRIAL))
# 48 rows with NAs for COVCLIN Trail 
df <- dfor %>% dplyr::filter(!is.na(COVCLINTRIAL))

#Print Dimensions
dim(df)

#Print Unique Record ID
cat("\n Complete COVCLINTRIAL Record, Patient, Hospital Counts")
length(unique(df$RCRDNUM)) 
length(unique(df$PATIENT_ID))
length(unique(df$SRC_FAC_ID))


[1] 8920  431

Original Record, Patient, Hospital Counts

[1] 8920

[1] 8592

[1] 88

...

[1] 8872  431


 Complete COVCLINTRIAL Record, Patient, Hospital Counts

[1] 8872

[1] 8545

[1] 86

In [13]:
8920-8545 #number of records with duplicate patient IDs
8920-48-431 #dimensions using Adriana's counts
8920-48-327 #dimensions using Laura's counts

[1] 375

[1] 8441

[1] 8545

In [28]:

## Identify Duplicates
##Searches onces from the top down and once from the bottom up.
##Adds "dups" variable to the df where duplicate is TRUE 
df$dups <- duplicated(pull(df, PATIENT_ID)) | duplicated(pull(df,PATIENT_ID),fromLast=TRUE)


#Extracts Duplicates where TRUE
dup_df <- df[df$dups == "TRUE", ]

#431 duplicates identified
dim(dup_df)


#Number of rows with at least one repeating PATIENT_ID and 
#number of unique repeating PATIENT_IDs row(dup_df)

length(unique(dup_df$PATIENT_ID))
dim(df[duplicated(df$PATIENT_ID, fromLast = TRUE),]) 
dim(df[duplicated(df$PATIENT_ID),]) 

dup_summary <-  dup_df %>% 
                group_by(PATIENT_ID) %>% 
                summarise(n_records = length(PATIENT_ID))

n_patients_per_dup_n  <-  dup_summary %>% 
                            group_by(n_records) %>% 
                            summarize(n_patients = n())
n_patients_per_dup_n 

#all records for patients with multiple records get filtered out,
#rather than keeping one record in

sum(df$PATIENT_ID[!df$dups] %in% dup_df$PATIENT_ID)

[1] 431 432

[1] 104

[1] 327 432

[1] 327 432

n_records,n_patients
<int>,<int>
2,96
3,4
6,1
7,1
18,1
196,1


[1] 0